In [87]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer

import string
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer #

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, accuracy_score, precision_score, recall_score, f1_score

# Model training/testing 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

#neural network
from keras.preprocessing.text import Tokenizer

In [88]:
df = pd.read_csv('LastOne.csv')
df
cols = {'Timestamp', 
        '1. Would you like to take part in this study?',
        '1. What gender do you identify as?',
        '2. How old are you?',
        '4. How many languages do you speak?',
        '2. What year did you graduate from University?',
        '8. What are some of the things you wish someone had told you prior to choosing this major?',
        '2. Do you feel motivated to go to work?',
        '1. If you had the courage would you quit?',
        '2. What would make you more motivated to continue?',
        '2. What is your level of education?',
        '3. What is your nationality?',
        '5. Where do you currently live?',
        
       }
df = df.drop(cols, axis = 1)
df = df.dropna()
df

,5. Did you graduate ?,6. What University did you attend?,1. What was your major of choice during your Bachelor's ?,Did you change your major during the course of your studies?,3. What made you choose this major?,What did you like the most about your major?,4. Are you currently working in your major field? Or in something related to it?,4. Do you like your job?,5. Do you regret your study field/major of choice?,6. If you had to make another major choice what would it be?,7. Would you advise others to make the same choice?,1. Are you satisfied with your job?
2,Yes,Lebanese American University,Computer science,No,Interest in the major,Being an introvert is not frowned upon,Yes,Yes,No,Prostitution,No,No
3,Yes,LAU,Computer Science,No,"Pressure (Peer, Society,...)",The mathematical and the problem solving aspect.,Yes,Yes,Maybe,Archeology,Yes,Yes
4,Yes,LAU,Civil engineering,No,Interest in the major,My position in the field,Yes,Yes,No,Business,Yes,Yes
5,Yes,LAU,Computer science,No,Interest in the major,"All the critical thinking that we need to to, ...",Maybe,Yes,No,"Psychology, or i would study a specific field ...",Yes,Maybe
6,Yes,Instituto de Empresa Universidad,Dual Bachelors in Business Administration and ...,No,Challenge/Intrigue,It has provided experience unlike anywhere I c...,Maybe,Yes,Maybe,Hard science/mathematics and econometrics,No,Maybe
...,...,...,...,...,...,...,...,...,...,...,...,...
218,Yes,AUST,Graphic design,No,Interest in the major,The market,Yes,No,Maybe,Psychology,Yes,No
219,Yes,LAU,Business administration,No,"Pressure (Peer, Society,...)",Nothing,Yes,Yes,Yes,Journalism .. archeology,No,Maybe
221,Yes,LAU,aa,Yes,"Pressure (Peer, Society,...)",aa,Yes,Yes,Yes,aa,Yes,No
222,Yes,LAU,aa,Yes,Assumed Financial Prosperity (Money),aa,Yes,Yes,Yes,aa,Yes,Yes


In [89]:
en_stopwords = stopwords.words("english")
lemma = WordNetLemmatizer()
ps = PorterStemmer()
#define a function for preprocessing
def clean(text):
    text = re.sub("[^A-Za-z1-9 ]", "", text) #removes punctuation marks
    # text = text.lower() #changes to lower case
    tokens = word_tokenize(text) #tokenize the text
    clean_list = [] 
    for token in tokens:
        if token not in en_stopwords: #removes stopwords
            clean_list.append(lemma.lemmatize(token)) #lemmatizing and appends to clean_list
    return " ".join(clean_list)# joins the tokens


def clean1(text):
    text =re.sub('/', '-'. text)
    return text

In [90]:
# df['Cleaned_0'] = df['3. What made you choose this major?'].apply(clean)
df['What did you like the most about your major?'] = df['What did you like the most about your major?'].apply(clean)
df['1. What was your major of choice during your Bachelor\'s ?'] = df['1. What was your major of choice during your Bachelor\'s ?'].str.lower()
df['1. What was your major of choice during your Bachelor\'s ?'] = df['1. What was your major of choice during your Bachelor\'s ?'].str.strip()

In [91]:
df.drop(df.loc[df['1. What was your major of choice during your Bachelor\'s ?']== 'mba'].index, inplace=True)
df.drop(df.loc[df['1. What was your major of choice during your Bachelor\'s ?']== 'emba'].index, inplace=True)
df.drop(df.loc[df['1. What was your major of choice during your Bachelor\'s ?']== 'enba'].index, inplace=True)
df.drop(df.loc[df['1. What was your major of choice during your Bachelor\'s ?']== 'global mba'].index, inplace=True)
df.drop(df.loc[df['1. What was your major of choice during your Bachelor\'s ?']== 'aa'].index, inplace=True)
df.drop(df.loc[df['1. What was your major of choice during your Bachelor\'s ?']== 'matlem'].index, inplace=True)
df.drop(df.loc[df['1. What was your major of choice during your Bachelor\'s ?']== 'executive mba'].index, inplace=True)
df['1. What was your major of choice during your Bachelor\'s ?'].values

array(['computer science', 'computer science', 'civil engineering',
       'computer science',
       'dual bachelors in business administration and international relations',
       'banking & finance',
       'interior architecture / environmental design',
       'international business', 'civil engineering',
       'interior architecture', 'civil', 'international business',
       'international business', 'communication', 'psychology',
       'civil engineering', 'computer science', 'business administration',
       'civil engineering', 'biology', 'mechanical engineering',
       'electrical eng', 'industrial engineer', 'finance', 'biology',
       'computer science', 'biology', 'it', 'business- finance',
       'pharmacy', 'nursing', 'mechanical engineering', 'economics',
       'economics', 'marketing', 'business marketing',
       'mechanical engineering', 'civil engineering', 'civil engineering',
       'civil engineering', 'computer science', 'llm in business law',
       'law'

In [92]:
df

,5. Did you graduate ?,6. What University did you attend?,1. What was your major of choice during your Bachelor's ?,Did you change your major during the course of your studies?,3. What made you choose this major?,What did you like the most about your major?,4. Are you currently working in your major field? Or in something related to it?,4. Do you like your job?,5. Do you regret your study field/major of choice?,6. If you had to make another major choice what would it be?,7. Would you advise others to make the same choice?,1. Are you satisfied with your job?
2,Yes,Lebanese American University,computer science,No,Interest in the major,Being introvert frowned upon,Yes,Yes,No,Prostitution,No,No
3,Yes,LAU,computer science,No,"Pressure (Peer, Society,...)",The mathematical problem solving aspect,Yes,Yes,Maybe,Archeology,Yes,Yes
4,Yes,LAU,civil engineering,No,Interest in the major,My position field,Yes,Yes,No,Business,Yes,Yes
5,Yes,LAU,computer science,No,Interest in the major,All critical thinking need logic need know cou...,Maybe,Yes,No,"Psychology, or i would study a specific field ...",Yes,Maybe
6,Yes,Instituto de Empresa Universidad,dual bachelors in business administration and ...,No,Challenge/Intrigue,It provided experience unlike anywhere I could...,Maybe,Yes,Maybe,Hard science/mathematics and econometrics,No,Maybe
...,...,...,...,...,...,...,...,...,...,...,...,...
216,Yes,LIU,computer science,No,Interest in the major,Wide range future opportunity,Yes,Yes,No,Graphic Design,Yes,Yes
217,Yes,LAU,law,No,Interest in the major,Helping People lawyer prestigious socially,No,Yes,Maybe,Business,Yes,Yes
218,Yes,AUST,graphic design,No,Interest in the major,The market,Yes,No,Maybe,Psychology,Yes,No
219,Yes,LAU,business administration,No,"Pressure (Peer, Society,...)",Nothing,Yes,Yes,Yes,Journalism .. archeology,No,Maybe


In [93]:
df =df.iloc[:,2:9]
df

,1. What was your major of choice during your Bachelor's ?,Did you change your major during the course of your studies?,3. What made you choose this major?,What did you like the most about your major?,4. Are you currently working in your major field? Or in something related to it?,4. Do you like your job?,5. Do you regret your study field/major of choice?
2,computer science,No,Interest in the major,Being introvert frowned upon,Yes,Yes,No
3,computer science,No,"Pressure (Peer, Society,...)",The mathematical problem solving aspect,Yes,Yes,Maybe
4,civil engineering,No,Interest in the major,My position field,Yes,Yes,No
5,computer science,No,Interest in the major,All critical thinking need logic need know cou...,Maybe,Yes,No
6,dual bachelors in business administration and ...,No,Challenge/Intrigue,It provided experience unlike anywhere I could...,Maybe,Yes,Maybe
...,...,...,...,...,...,...,...
216,computer science,No,Interest in the major,Wide range future opportunity,Yes,Yes,No
217,law,No,Interest in the major,Helping People lawyer prestigious socially,No,Yes,Maybe
218,graphic design,No,Interest in the major,The market,Yes,No,Maybe
219,business administration,No,"Pressure (Peer, Society,...)",Nothing,Yes,Yes,Yes


In [94]:
df

,1. What was your major of choice during your Bachelor's ?,Did you change your major during the course of your studies?,3. What made you choose this major?,What did you like the most about your major?,4. Are you currently working in your major field? Or in something related to it?,4. Do you like your job?,5. Do you regret your study field/major of choice?
2,computer science,No,Interest in the major,Being introvert frowned upon,Yes,Yes,No
3,computer science,No,"Pressure (Peer, Society,...)",The mathematical problem solving aspect,Yes,Yes,Maybe
4,civil engineering,No,Interest in the major,My position field,Yes,Yes,No
5,computer science,No,Interest in the major,All critical thinking need logic need know cou...,Maybe,Yes,No
6,dual bachelors in business administration and ...,No,Challenge/Intrigue,It provided experience unlike anywhere I could...,Maybe,Yes,Maybe
...,...,...,...,...,...,...,...
216,computer science,No,Interest in the major,Wide range future opportunity,Yes,Yes,No
217,law,No,Interest in the major,Helping People lawyer prestigious socially,No,Yes,Maybe
218,graphic design,No,Interest in the major,The market,Yes,No,Maybe
219,business administration,No,"Pressure (Peer, Society,...)",Nothing,Yes,Yes,Yes


In [95]:
majors_pd = pd.read_csv('Majors.csv')
majors_df = df.iloc[:,0]
key = majors_pd.iloc[:,1]
majors_df


2                                       computer science
3                                       computer science
4                                      civil engineering
5                                       computer science
6      dual bachelors in business administration and ...
                             ...                        
216                                     computer science
217                                                  law
218                                       graphic design
219                              business administration
223                                           ba-finance
Name: 1. What was your major of choice during your Bachelor's ?, Length: 149, dtype: object

In [96]:
for i in majors_df:
  # print(i)
  l = majors_pd[majors_pd['Majors']== i].index.tolist()
  p = df[df['1. What was your major of choice during your Bachelor\'s ?']== i].index.tolist()
  df.loc[p,'New_Majors_Category'] = key._get_value(l[0])
  # print(l , p)

C:\Users\user\AppData\Local\Temp\ipykernel_13960\3837046732.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[p,'New_Majors_Category'] = key._get_value(l[0])


In [97]:
df['New_Majors_Category'] = df['New_Majors_Category'].astype(int)
df

C:\Users\user\AppData\Local\Temp\ipykernel_13960\4026744212.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['New_Majors_Category'] = df['New_Majors_Category'].astype(int)


,1. What was your major of choice during your Bachelor's ?,Did you change your major during the course of your studies?,3. What made you choose this major?,What did you like the most about your major?,4. Are you currently working in your major field? Or in something related to it?,4. Do you like your job?,5. Do you regret your study field/major of choice?,New_Majors_Category
2,computer science,No,Interest in the major,Being introvert frowned upon,Yes,Yes,No,1
3,computer science,No,"Pressure (Peer, Society,...)",The mathematical problem solving aspect,Yes,Yes,Maybe,1
4,civil engineering,No,Interest in the major,My position field,Yes,Yes,No,0
5,computer science,No,Interest in the major,All critical thinking need logic need know cou...,Maybe,Yes,No,1
6,dual bachelors in business administration and ...,No,Challenge/Intrigue,It provided experience unlike anywhere I could...,Maybe,Yes,Maybe,14
...,...,...,...,...,...,...,...,...
216,computer science,No,Interest in the major,Wide range future opportunity,Yes,Yes,No,1
217,law,No,Interest in the major,Helping People lawyer prestigious socially,No,Yes,Maybe,17
218,graphic design,No,Interest in the major,The market,Yes,No,Maybe,35
219,business administration,No,"Pressure (Peer, Society,...)",Nothing,Yes,Yes,Yes,11


In [98]:
X = df.iloc[:, 1:6]
X

,Did you change your major during the course of your studies?,3. What made you choose this major?,What did you like the most about your major?,4. Are you currently working in your major field? Or in something related to it?,4. Do you like your job?
2,No,Interest in the major,Being introvert frowned upon,Yes,Yes
3,No,"Pressure (Peer, Society,...)",The mathematical problem solving aspect,Yes,Yes
4,No,Interest in the major,My position field,Yes,Yes
5,No,Interest in the major,All critical thinking need logic need know cou...,Maybe,Yes
6,No,Challenge/Intrigue,It provided experience unlike anywhere I could...,Maybe,Yes
...,...,...,...,...,...
216,No,Interest in the major,Wide range future opportunity,Yes,Yes
217,No,Interest in the major,Helping People lawyer prestigious socially,No,Yes
218,No,Interest in the major,The market,Yes,No
219,No,"Pressure (Peer, Society,...)",Nothing,Yes,Yes


In [99]:
y = df.iloc[:,7]
y

2       1
3       1
4       0
5       1
6      14
       ..
216     1
217    17
218    35
219    11
223     4
Name: New_Majors_Category, Length: 149, dtype: int32

In [100]:
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder(handle_unknown='ignore',sparse=False)

X = pd.DataFrame(ohe.fit_transform(X))

X

,0,1,2,3,4,5,6,7,8,9,...,148,149,150,151,152,153,154,155,156,157
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
4,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
145,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
146,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
147,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0


In [101]:
from sklearn.naive_bayes import MultinomialNB

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.05, random_state=123)

model = MultinomialNB()
model.fit(X_train, y_train)

model.score(X_test, y_test)

0.125

In [102]:
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier

#List Hyperparameters that we want to tune.
leaf_size = list(range(1,50))
n_neighbors = list(range(1,30))
p=[1,2]
#Convert to dictionary
hyperparameters = dict(leaf_size=leaf_size, n_neighbors=n_neighbors, p=p)
#Create new KNN object
knn_2 = KNeighborsClassifier()
#Use GridSearch
clf = GridSearchCV(knn_2, hyperparameters, cv=10)
#Fit the model
best_model = clf.fit(X_train,y_train)

c:\Users\user\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  warnings.warn(


In [110]:
X_train

,0,1,2,3,4,5,6,7,8,9,...,148,149,150,151,152,153,154,155,156,157
8,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
90,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
140,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
33,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
59,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
98,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
66,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
126,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0


In [109]:
X_test

,0,1,2,3,4,5,6,7,8,9,...,148,149,150,151,152,153,154,155,156,157
72,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
112,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
131,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
88,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
37,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
125,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
87,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
42,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0


In [103]:
y_pred = clf.predict(X_test)

In [104]:
from sklearn.metrics import accuracy_score

accuracy_score(y_pred,y_test)

0.0

In [105]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=1)
knn.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=1)

In [106]:
y_pred1 = knn.predict(X_test)
y_pred1

array([ 4,  6,  1, 24,  1,  8,  4,  1])

In [107]:
from sklearn.metrics import accuracy_score

accuracy_score(y_pred1,y_test)

0.125

In [114]:
sample_dataset = [['No','Interest in the major', 'I like problem solving', 'Yes', 'Yes']]


knn.predict(ohe.transform(sample_dataset))

c:\Users\user\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


array([28])